In [10]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [9]:
device = torch.device('cuda')

In [8]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
train_dataset = torchvision.datasets.MNIST(root = '../../data',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)
test_dataset = torchvision.datasets.MNIST(root = '../../data',
                                          train = False,
                                          transform = transforms.ToTensor())

In [7]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [11]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
        

In [12]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)


In [14]:
total = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images= images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3526
Epoch [1/5], Step [200/600], Loss: 0.2495
Epoch [1/5], Step [300/600], Loss: 0.1692
Epoch [1/5], Step [400/600], Loss: 0.1190
Epoch [1/5], Step [500/600], Loss: 0.2331
Epoch [1/5], Step [600/600], Loss: 0.2408
Epoch [2/5], Step [100/600], Loss: 0.1191
Epoch [2/5], Step [200/600], Loss: 0.1492
Epoch [2/5], Step [300/600], Loss: 0.1530
Epoch [2/5], Step [400/600], Loss: 0.0829
Epoch [2/5], Step [500/600], Loss: 0.1167
Epoch [2/5], Step [600/600], Loss: 0.0675
Epoch [3/5], Step [100/600], Loss: 0.1041
Epoch [3/5], Step [200/600], Loss: 0.0482
Epoch [3/5], Step [300/600], Loss: 0.0290
Epoch [3/5], Step [400/600], Loss: 0.1786
Epoch [3/5], Step [500/600], Loss: 0.0664
Epoch [3/5], Step [600/600], Loss: 0.0469
Epoch [4/5], Step [100/600], Loss: 0.0278
Epoch [4/5], Step [200/600], Loss: 0.0343
Epoch [4/5], Step [300/600], Loss: 0.0425
Epoch [4/5], Step [400/600], Loss: 0.0866
Epoch [4/5], Step [500/600], Loss: 0.0154
Epoch [4/5], Step [600/600], Loss:

In [22]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: {}%'.format(100*correct/total))


Accuracy of the network on the 10000 test images: 97.96%


In [23]:
torch.save(model.state_dict(),'model.ckpt')